<a href="https://colab.research.google.com/github/ucfilho/optim_raianars/blob/master/ABCOptim_jan_02_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from random import randint

In [0]:
'''Rosembrock Function'''
def Fun(x):
    fun=0
    a=1.0
    b=100.0
    fun = (a-x[0])**2 + b*(x[1]-x[0]**2)**2
    return fun

In [0]:
''' 
FOBJ gera vetorialmente os valores com a funcao objetivo escalar
em outras palavras gera para populacao o valor da funcao 
a ser otimizada
'''
def FOBJ(X):
    rows = X.shape[0]
    fobj=np.zeros(rows)
    for i in range(rows):
        fobj[i]=Fun(X[i,])
    return fobj

In [0]:
'''Enxame retorna a populacao aleatoria com todos valores entre MIN e MAX'''
def Enxame(PAR,NPAR,MAX,MIN):
    x=np.zeros((NPAR, len(MAX)))
    for j in range(len(MAX)):
        for i in range(NPAR):
            x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x

In [0]:
''' 
Fitness gera vetorialmente os valores com a funcao fitness
equivale a quantidade de nectar da fonte
'''
def FIT(X):
    rows = X.shape[0]
    fit=np.zeros(rows)
    fit=np.copy(FOBJ(X))
    for i in range(rows):
        if(fit[i]>=0):
            fit[i]=1/(fit[i]+1)
        else:
            fit[i]=1+abs(fit[i])
    return fit

In [0]:
'''
probablilidade usada para roleta q atualizacao das onlookers 
semelhante ao genetico
'''

def PROB(x):
    rows=x.shape[0]
    fit=np.zeros(rows)
    prob=np.zeros(rows)
    fit=FIT(x)
    soma=np.sum(fit)
    
    for i in range(rows):
        prob[i] = fit[i]/soma

    return prob

In [0]:
'''
Obtem a melhor escolha greedy search
'''
def GetBest(x,xbest):
    ycal=FOBJ(x)
    best=np.argmin(ycal)
    yref=Fun(xbest)
    if(yref<ycal[best]):
        GBEST=xbest
    else:
        GBEST=x[best,]
    return GBEST
    

In [0]:
'''
employed bee phase
'''
def EmployedBee(xo,x) :
    NPAR = xo.shape[0]
    PAR= xo.shape[1]
    V=np.zeros((NPAR, len(MAX)))

    for j in range(PAR):
        for i in range(NPAR):
            fi=np.random.uniform(low=-1.0, high=1.0, size=None)
            rd=randint(0, (NPAR-1))
            V[i,j]=xo[i,j]+(xo[i,j] -x[rd,j])*fi # candidata a solucao
    YCAL=FOBJ(xo)
    YV=FOBJ(V)
    for i in range(NPAR):
        if(YV[i]<YCAL[i]):
            x[i,]=np.copy(V[i,])

    return x

In [0]:
'''
onlooker bee phase
'''
def OnlookerBee(xo,x):
    rows = xo.shape[0]
    cols = xo.shape[1]
    prob=PROB(x)
    best=np.argmax(prob)
    
    for i in range(rows):
        if(Fun(x[best,])<Fun(xo[i,])):
            xo[i,]=np.copy(x[best,])
    return xo

In [0]:
'''
scout bee phase
'''
#def ScoutBee(trial,x,ntrail,MAX,MIN):
def ScoutBee(x,MAX,MIN):
    PAR = x.shape[0]
    NPAR = x.shape[1]
    for i in range(PAR):
        for j in range(NPAR):
            x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x

In [0]:

'''
Metodo que alterna e coordena o uso employers,onlooker, scout bees
e greedy search

'''
def ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN):

    trial=0 # inicializa contador p abandonar fonte de alimento
    xbest=Enxame(PAR,1,MAX,MIN)[0,] #inicializa xbest
    xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee-parte 1

    for i in range(ITE):
        x=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee-parte 1
        
        x_Employed=EmployedBee(xo,x)
        xbest=GetBest(x_Employed,xbest)
        xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa OnlookerBee
        x_Onlooker=OnlookerBee(xo,x_Employed)
        xbest=GetBest(x_Onlooker,xbest)
        xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa ScoutBee ( a rigor so esta linha basta)
        x_Scout=ScoutBee(xo,MAX,MIN) # a rigor so a linha acima basta esta linha eh para futura melhora
        xbest=GetBest(x_Scout,xbest)
    return xbest  

In [0]:
NPAR=30 #PARTICULAS
ITE=300 #ITERACOES
PAR=2 #NUM DE PARAMETROS A SER OTIMIZADOS
MAX=[2,2] # MAXIMO DE CADA PARAMETRO
MIN=[-2,-2] # MINIMO DE CADA PARAMETRO
ntrail=50 #numero de buscas ate abandonar uma fonte de alimento

for i in range(6):
    print("resolucao",i+1," ")
    xbest=ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN)
    print("vetor",xbest,"funcao", Fun(xbest),"\n")

resolucao 1  
vetor [1.05586673 1.11169288] funcao 0.0041207059552531835 

resolucao 2  
vetor [0.98543537 0.97210544] funcao 0.00031669573968676646 

resolucao 3  
vetor [0.99186902 0.98146123] funcao 0.0006150466133226075 

resolucao 4  
vetor [0.97877024 0.95649402] funcao 0.000674851025990242 

resolucao 5  
vetor [0.96522566 0.93124467] funcao 0.0012265522146899476 

resolucao 6  
vetor [0.9998236  1.00254175] funcao 0.0008378487931252882 

